<a href="https://colab.research.google.com/github/marimarmatos/AnyoneAI/blob/main/21_1_PRACTICE_Introduction_to_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Practice

This notebook provides a hands-on opportunity to practice prompt engineering, a crucial skill for effectively interacting with large language models (LLMs). Crafting clear, concise, and well-structured prompts is essential for obtaining desired outputs and unlocking the full potential of LLMs.

Mastering prompt engineering is fundamental for more advanced LLM applications and training methodologies, such as Reinforcement Learning from Human Feedback (RLHF). By understanding how to guide LLMs through prompt design, you lay the groundwork for fine-tuning models and aligning their behavior with specific goals and desired outcomes.

In [1]:
# Let's start by setting the API key
import json
import openai

# Read the API key from a JSON file
with open('api_gpt.json', 'r') as file:
    config = json.load(file)

openai.api_key = config['api_key']

## First Task
Generate a simple system prompt that takes the name of an algorithm, and asks for a response that should only contain code with several test case.

In [6]:
# Put an algorithm name here
algorithm_name = "count of dogs in DataFrames"
# Create the system prompt
system_prompt = f"""
You are an expert in programming.
Given the name of an algorithm, you will respond solely with the implementation code of that algorithm,
including several test cases to demonstrate its functionality.
Do not include any explanations or additional text.

Algorithm Name:
{algorithm_name}
"""



In [7]:
# Call the model
completion = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": system_prompt.format(algorithm_name)}
  ]
)

print(completion.choices[0].message.content)

```python
import pandas as pd

def count_dogs(df: pd.DataFrame) -> int:
    return df['animal'].str.lower().str.count('dog').sum()

# Test cases
data1 = {'animal': ['Dog', 'Cat', 'fish', 'dog', 'dog', 'Mouse']}
df1 = pd.DataFrame(data1)
print(count_dogs(df1))  # Output: 3

data2 = {'animal': ['dog', 'dog', 'Dog', 'DOG', 'cat']}
df2 = pd.DataFrame(data2)
print(count_dogs(df2))  # Output: 4

data3 = {'animal': ['Cat', 'Fish', 'Hamster']}
df3 = pd.DataFrame(data3)
print(count_dogs(df3))  # Output: 0

data4 = {'animal': ['Dog', 'Dog', 'cat', 'dog', 'Tiger', 'DOG']}
df4 = pd.DataFrame(data4)
print(count_dogs(df4))  # Output: 4

data5 = {'animal': []}
df5 = pd.DataFrame(data5)
print(count_dogs(df5))  # Output: 0
```


## Second Task
Situation: You're curating a dataset containing prompts that ask for Image Generation tasks. However, the dataset contains prompts that are not usable for the project that you're working on right now. Currently, you want only non ambigous and human-like prompts. To help you classify the prompts, you want to create a system template that gives you a score between 0 and 1 (with 0 being an unuseful prompt and 1 a perfect prompt) and a brieft explanation.

### Definitions:
Unambigous: The prompts needs to explicitly mention figures and some visual details.

Humanlike: The prompt should sound human, containing a some personality, not just a very robotic list of requirements

In [17]:
# Here are some examples:
# Ambigous prompts
ambiguous_prompt_1 = "Create an image of a city."
ambiguous_prompt_2 = "Generate an artwork of a park setting."

# Non-Humanlike Prompts
non_humanlike_prompt_1 = "Illustrate a scene: Elephant under tree, sunny backdrop, time: midday, color: realistic."
non_humanlike_prompt_2 = "Render image: Two children playing with ball in park, include details: bench, kite in sky, day background."

# Perfect prompts
perfect_prompt_1 = "Imagine a cozy reading nook with a plush armchair, a small wooden side table, and a window with morning sunlight filtering in through soft, sheer curtains. Could you illustrate this warm, inviting scene?"
perfect_prompt_2 = "I'd love to see a serene beach at sunset, with gentle waves lapping at the shore and a lone seagull gliding across a vibrant orange and pink sky. Can you create this tranquil moment?"

# Task: Create yourself two prompts that would be perfect
your_prompt_1 = "create a sunset at the beach with orange and pink colors, two mountains on either side of the horizon and a fishing boat on the left side of the image"
your_prompt_2 = "create an orange kitten using a blue toy cap"

In [18]:
system_prompt = """
You are an expert evaluator in classifying prompts for image generation, responsible for determining how useful they are for a specific project. For each prompt provided, you must do the following:

Rating: Assign a score of 0 or 1, not decimals, only scores 0 or 1, where:

0 indicates that the prompt is not useful, ambiguous, or irrelevant for image generation.
1 indicates that the prompt is excellent, clear, specific, and appropriate for generating high-quality images.
Explanation: Provide a brief justification in 2 or 3 sentences explaining why you assigned that particular score, highlighting positive or negative aspects based on the following criteria:

Unambiguity: Does the prompt explicitly mention figures, objects, or visual details? Useful prompts should clearly describe what needs to be generated with concrete details.
Humanlike/Personality: Does the prompt sound natural and human? It should have a conversational tone, include nuances or characteristics that reflect human intent or style rather than mechanical or cold lists.
Clarity and Precision: Is the prompt clear, unambiguous, and with well-defined visual expectations? Precise prompts help achieve more suitable and coherent results.
Response: Only provide the score (a decimal between 0 and 1) and the brief explanation, without additional comments or debates.

Here goes the prompt to evaluate:
{}
"""

In [15]:
print(system_prompt.format(ambiguous_prompt_1))


You are an expert evaluator in classifying prompts for image generation, responsible for determining how useful they are for a specific project. For each prompt provided, you must do the following:

Rating: Assign a score of 0 or 1, where:

0 indicates that the prompt is not useful, ambiguous, or irrelevant for image generation.
1 indicates that the prompt is excellent, clear, specific, and appropriate for generating high-quality images.
Explanation: Provide a brief justification in 2 or 3 sentences explaining why you assigned that particular score, highlighting positive or negative aspects based on the following criteria:

Unambiguity: Does the prompt explicitly mention figures, objects, or visual details? Useful prompts should clearly describe what needs to be generated with concrete details.
Humanlike/Personality: Does the prompt sound natural and human? It should have a conversational tone, include nuances or characteristics that reflect human intent or style rather than mechanica

In [19]:
# Create an autograder function
def call_autograder(prompt):
  completion = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}

    ]
  )
  return completion.choices[0].message.content

# Call the model with each prompt to evaluate the results
prompt_list = [ambiguous_prompt_1,
              ambiguous_prompt_2,
              non_humanlike_prompt_1,
              non_humanlike_prompt_2,
              perfect_prompt_1,
              perfect_prompt_2,
              your_prompt_1,
              your_prompt_2]

for prompt_to_evaluate in prompt_list:
  full_prompt = system_prompt.format(prompt_to_evaluate)
  result = call_autograder(full_prompt)
  print(result)

0: The prompt is too vague and lacks specific details about the city that should be depicted, such as architectural style, time of day, or any distinctive features. Without clearer visual expectations or descriptions, it makes it difficult to generate a meaningful and coherent image.
0  
The prompt is too vague and lacks specific details about the park setting, such as the type of park, time of day, season, or any particular elements to include. This ambiguity may lead to varied interpretations, resulting in an unclear or unsatisfactory image.
1: The prompt is clear and specific, explicitly detailing the elements to include in the image, such as the elephant, tree, and sunny backdrop. It provides a well-defined visual expectation with a realistic color scheme, making it useful for generating a high-quality image.
1  
The prompt is clear and specific, providing explicit details about the scene including the figures (two children), objects (ball, bench, kite), and setting (park, day back

With Gemini API

In [20]:
# Import the Python SDK
import google.generativeai as genai
import json

# Read the API key from a JSON file
with open('api_google.json', 'r') as file:
    config = json.load(file)



genai.configure(api_key=config['api_key'])

# Initialize the Gemini API
gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')

Task 1

In [21]:
# Put an algorithm name here
algorithm_name = "count of dogs in DataFrames"
# Create the system prompt
system_prompt = f"""
You are an expert in programming.
Given the name of an algorithm, you will respond solely with the implementation code of that algorithm,
including several test cases to demonstrate its functionality.
Do not include any explanations or additional text.

Algorithm Name:
{algorithm_name}
"""

In [22]:
# Call the model with the new prompt
response = gemini_model.generate_content(system_prompt)
print(response.text)

```python
import pandas as pd

def count_dogs(df, dog_column):
    try:
        return df[dog_column].value_counts().get('dog', 0)
    except KeyError:
        return 0


#Test Cases
data1 = {'animal': ['dog', 'cat', 'dog', 'bird']}
df1 = pd.DataFrame(data1)
print(count_dogs(df1, 'animal')) #Output: 2

data2 = {'pet': ['cat', 'cat', 'bird']}
df2 = pd.DataFrame(data2)
print(count_dogs(df2, 'pet')) # Output: 0

data3 = {'animal': ['dog', 'dog', 'dog']}
df3 = pd.DataFrame(data3)
print(count_dogs(df3, 'animal')) #Output:3

data4 = {'animal': [1,2,3]}
df4 = pd.DataFrame(data4)
print(count_dogs(df4, 'animal')) # Output: 0

data5 = {}
df5 = pd.DataFrame(data5)
print(count_dogs(df5, 'animal')) # Output: 0

data6 = {'animal': ['dog', 'cat', 'Dog', 'DOG']}
df6 = pd.DataFrame(data6)
print(count_dogs(df6, 'animal')) # Output: 1

data7 = {'animal': ['dog', 'cat', 'dog', 'bird', None]}
df7 = pd.DataFrame(data7)
print(count_dogs(df7, 'animal')) # Output: 2


```



Task 2